# Adding paths and files

In [1]:
import os
import glob
import polars as pl

# Specify the directory where your CSV files are located
directory = r'D:\github\localt20\data\t20s_csv2'

# Use glob to find all CSV files in the specified directory
info_files = glob.glob(os.path.join(directory, '*_info.csv'))
all_files = glob.glob(os.path.join(directory,'*.csv'))
delivery_files = [file for file in all_files if '_info' not in file]

matches=[]
deliveries=[]
# Print the list of CSV files
for info_file in info_files:
    matches.append(info_file.split('\\')[-1])
for delivery in delivery_files:
    if '_info' not in delivery:
        deliveries.append(delivery.split('\\')[-1])

In [2]:
match_ids=[]
for csv_file in matches:
    match_ids.append(csv_file.split('_')[0])
    
# Define the initial and final schemas
initial_schema = {'col1': pl.String, 'attributes': pl.String, 'values': pl.String, 'players': pl.String, 'code': pl.String}
final_schema = [
    ('team1', pl.String),
    ('team2', pl.String),
    ('gender', pl.String),
    ('season', pl.String),
    ('date', pl.String),
    ('venue', pl.String),
    ('city', pl.String),
    ('toss_winner', pl.String),
    ('toss_decision', pl.String),
    ('winner', pl.String),
]

# Create a dictionary from the final schema
final_schema_dict = {key: value for key, value in final_schema}

# Initialize an empty DataFrame with the final schema
matches_data = pl.DataFrame(schema=final_schema_dict)

# List to store recalculated match IDs
recalculated_matchids = match_ids[:]

# Iterate over matches and process each one
for idx, match in enumerate(matches):
    try:
        match_df = pl.read_csv(f'../../data/t20s_csv2/{match}', schema=initial_schema)
        
        # Extract team names
        team1_name = match_df[1, 'values']
        team2_name = match_df[2, 'values']
        
        # Replace team names
        match_df = match_df.with_columns([
            pl.when((pl.col('attributes') == 'team') & (pl.col('values') == team1_name))
            .then(pl.lit('team1'))
            .when((pl.col('attributes') == 'team') & (pl.col('values') == team2_name))
            .then(pl.lit('team2'))
            .otherwise(pl.col('attributes'))
            .alias('attributes')
        ])
        
        # Select and transpose the DataFrame
        match_transposed = match_df.select("attributes", "values").transpose(include_header=True, column_names="attributes").drop("column")
        
        # Ensure all columns in final_schema_dict are present
        missing_cols = [col for col in final_schema_dict.keys() if col not in match_transposed.columns]
        if missing_cols:
            raise ValueError(f"Missing columns: {missing_cols}")

        # Select the required columns and append to matches_data
        match_transposed = match_transposed.select(final_schema_dict.keys())
        matches_data = matches_data.vstack(match_transposed)
    except Exception as e:
        recalculated_matchids.remove(match_ids[idx])
matches_data=matches_data.with_columns(pl.Series(recalculated_matchids).alias("match_id").cast(pl.Int64))
matches_data

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
str,str,str,str,str,str,str,str,str,str,i64
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/17""","""Melbourne Cricket Ground""",null,"""Sri Lanka""","""field""","""Sri Lanka""",1001349
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/19""","""Simonds Stadium, South Geelong""","""Victoria""","""Sri Lanka""","""field""","""Sri Lanka""",1001351
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/22""","""Adelaide Oval""",null,"""Sri Lanka""","""field""","""Australia""",1001353
"""Ireland""","""Hong Kong""","""male""","""2016""","""2016/09/05""","""Bready Cricket Club, Magherama…","""Londonderry""","""Hong Kong""","""bat""","""Hong Kong""",1004729
"""Zimbabwe""","""India""","""male""","""2016""","""2016/06/18""","""Harare Sports Club""",null,"""India""","""field""","""Zimbabwe""",1007655
…,…,…,…,…,…,…,…,…,…,…
"""Pakistan""","""Sri Lanka""","""male""","""2015/16""","""2016/03/04""","""Shere Bangla National Stadium""","""Mirpur""","""Pakistan""","""field""","""Pakistan""",966763
"""Bangladesh""","""India""","""male""","""2015/16""","""2016/03/06""","""Shere Bangla National Stadium""","""Mirpur""","""India""","""field""","""India""",966765
"""United Arab Emirates""","""Netherlands""","""male""","""2015/16""","""2016/02/03""","""ICC Academy""","""Dubai""","""United Arab Emirates""","""field""","""Netherlands""",967081


In [5]:
matches_data['team1'].unique().sort().to_list()

['Afghanistan',
 'Argentina',
 'Australia',
 'Austria',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belgium',
 'Belize',
 'Bermuda',
 'Bhutan',
 'Botswana',
 'Brazil',
 'Bulgaria',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Chile',
 'China',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'England',
 'Estonia',
 'Eswatini',
 'Fiji',
 'Finland',
 'France',
 'Gambia',
 'Germany',
 'Ghana',
 'Gibraltar',
 'Greece',
 'Guernsey',
 'Hong Kong',
 'Hungary',
 'India',
 'Indonesia',
 'Iran',
 'Ireland',
 'Isle of Man',
 'Israel',
 'Italy',
 'Japan',
 'Jersey',
 'Kenya',
 'Kuwait',
 'Lesotho',
 'Luxembourg',
 'Malawi',
 'Malaysia',
 'Maldives',
 'Mali',
 'Malta',
 'Mexico',
 'Mongolia',
 'Mozambique',
 'Myanmar',
 'Namibia',
 'Nepal',
 'Netherlands',
 'New Zealand',
 'Nigeria',
 'Norway',
 'Oman',
 'Pakistan',
 'Panama',
 'Papua New Guinea',
 'Philippines',
 'Portugal',
 'Qatar',
 'Romania',
 'Rwanda',
 'Samoa',
 'Saudi Arabia',
 'S

In [3]:
matches_data.null_count()

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,192,0,0,0,0


In [6]:
matches_data.write_csv('../../processedData/matches.csv')